In [ ]:
from cppbridge import *
from mcmc_analytics import *

In [ ]:
%load_ext autoreload
%autoreload 2

# Inferring parameters with EHMM samplers for Poisson model 2

The experiments are for the models and samplers presented in the paper\
[1] Shestopaloff A.Y, Neal R.M. Sampling Latent States for High-Dimensional Non-Linear State Space Models with the Embedded HMM Method

---
## Model specification
Here we will try to recover the parameters of the same non-linear SSM we used in the experiments with Poisson model 2 with known parameters (see `sampling_bimodal.ipynb` notebook). To do that we will specify the priors for each of the parameters driving the model: $a_i$ and $\rho$ for transition model and $\sigma_i$ for the observation model.

Out chosen priors are as follows:
* $a_{i}\sim\mathrm{U}(-1,1)$: for VAR(1) process we need our coefficients to be between -1 and 1
* $\rho\sim\mathrm{U}(0,1)$
* $\sigma_{i}\sim\mathcal{N}(0,\sigma^{2})$
where $\sigma^{2}$ is a large number to make the prior vague

In [ ]:
# Set up the transition model with unknown parameters
T = 250
n = 10
A = DiagonalMatrixParam()
aa = -1.
bb = 1.
A.parametrize(n, DistributionType.UNIFORM, (aa, bb), minx=aa, maxx=bb)
Q = SymmetricMatrixParam()
a = 0.
b = 1.
x_var = 1.
Q.parametrize(n, DistributionType.UNIFORM, (a, b), x_var, minx=a, maxx=b)
# Prior mean is always zero
prior_mean = np.zeros(n)
# The prior covariance will be re-calculated for each newly accepted parameters, including for the first iteration 
# so below will not be used but still required to intialize the class 
Q_init = np.eye(n)
trm = TransitionSpec(A, Q, prior_mean, Q_init)

In [ ]:
# Set up the observation model with unknown parameters
sigma = VectorParam()
mu = 0
var = 25
sigma.parametrize(n, DistributionType.NORMAL, (mu, np.sqrt(var)))
obsm = ObservationSpec(ModelType.BIMODAL_POISSON, sigma)

## Sampler specification
### Embedded HMM sampling scheme
In these experiments we will only be using EHMM sampler.

In [ ]:
# Specify EHMM scheme
nupd = 50  # number of parameter updates between the scheme runs
pool_sz = 80
ehmm_sampler = SamplerSpec(SamplerType.EHMM, pool_size=pool_sz, num_parameter_updates=nupd, flip=True)

## Observations
We need the observations, on which to run the samplers. The data we use is the synthetic data, generated in `models.ipynb` notebook. In this experiemtns we will try and recover the parameters used to generate that data as well as the latent states.

In [ ]:
model = "gauss_poiss2_3225125971228064704"
dataprovider = model + "_data.h5"
dataspec = Data(dataprovider)

## Simulation    
To run the simulation we need to provide the initial sample $\mathbf{x}_0$ to start off the sampler, and specify the parameters of the simulation. As in the case when we ran the model with known parameters, here we also set $\mathbf{x}_0=\mathbf{1}$ and run 5 simulations for $10 000$ iterations each, starting with the different seed for randomisation.

In [ ]:
n_iter_ehmm = 10000
x_init = np.ones((n, T))
seeds = np.array([1, 10, 100, 1e4, 1e5], dtype=int)
scales_ehmm = np.array([0.05, 0.2])
reverse = False
flip = True
simulation_ehmm = SimulationSpec(n_iter_ehmm, seeds, x_init, scaling=scales_ehmm, reverse=reverse)

In [ ]:
# Simulation with EHMM sampler
ehmm_session_name = f"param_ehmm80_flip_{model}"
mcmc_ehmm_bp = MCMCsession(ehmm_session_name)
if mcmc_ehmm_bp.hasResults():
    mcmc_ehmm_bp.loadResults()
else:
    mcmc_ehmm_bp.init(T, trm, obsm, ehmm_sampler, simulation_ehmm, dataspec)
    mcmc_ehmm_bp.run()

---
## Analysis of the results
### Overview

In [ ]:
ehmm_burnin = int(0.1 * 2 * (n_iter_ehmm + 1))  # <-- as we run on reversed sequence, too, there will be twice as many samples
ehmm_samples = mcmc_ehmm_bp.get_samples(ehmm_burnin)

In [ ]:
ac_ehmm = getACF(mcmc_ehmm_bp.samples, ehmm_samples)
taus_ehmm = 1 + 2 * np.sum(ac_ehmm, axis=2)
meantaus_ehmm = np.mean(taus_ehmm)

In [ ]:
ehmm_met_acc = 100 * np.mean(
    [acc[:T].mean() / (n_iter_ehmm * pool_sz * (1 + reverse)) for _, acc in mcmc_ehmm_bp.acceptances.items()])
ehmm_shift_acc = 100 * np.mean(
    [acc[T + 1:].mean() / (n_iter_ehmm * pool_sz * (1 + reverse)) for _, acc in mcmc_ehmm_bp.acceptances.items()])
ehmm_partrm_acc = 100 * np.mean([acc['trm'] / (n_iter_ehmm * nupd * (1 + reverse)) for _, acc in mcmc_ehmm_bp.param_acceptances.items()])
ehmm_parobm_acc = 100 * np.mean([acc['obsm'] / (n_iter_ehmm * nupd * (1 + reverse)) for _, acc in mcmc_ehmm_bp.param_acceptances.items()])
ehmm_overview = {"Num seeds": len(seeds),
                 "Num iter": n_iter_ehmm,
                 "Time per sample, ms": np.mean(list(mcmc_ehmm_bp.durations.values())) /
                                        list(mcmc_ehmm_bp.samples.values())[0].shape[0],
                 "Acceptance rate, autoregressive update, %": ehmm_met_acc,
                 "Acceptance rate, shift update, %": ehmm_shift_acc,
                 "Acceptance rate, param updates for transition model, %": ehmm_partrm_acc,
                 "Acceptance rate, param updates for observation model, %": ehmm_parobm_acc,
                 "Average autocorrelation time": meantaus_ehmm
                 }
overview = pd.DataFrame({"EHMM on Poisson model 2": ehmm_overview})
overview

In [ ]:
ehmm_ar = np.stack(list(mcmc_ehmm_bp.acceptances.values()), axis=1)[:T].ravel() / (
            n_iter_ehmm * pool_sz * (1 + reverse))
ehmm_shift = np.stack(list(mcmc_ehmm_bp.acceptances.values()), axis=1)[T + 1:].ravel() / (
            n_iter_ehmm * pool_sz * (1 + reverse))
pd.DataFrame(dict(ehmm_ar=ehmm_ar, ehmm_shift=np.append(ehmm_shift, [np.nan] * len(seeds)))).describe()

### Test equality of parameters

In [ ]:
params_names = ["$a_i$", "$\\rho$", "$\sigma_i$"]
true_values = dict(zip(params_names, [0.9, 0.7, 0.8]))
psamples = mcmc_ehmm_bp.get_paramSamples(ehmm_burnin, params_names)
params_ttest = ttest(psamples, list(true_values.values()))
pd.DataFrame({"p-value":params_ttest}, index=params_names)

In [ ]:
for pname in params_names:
    fig, ax = plt.subplots(figsize=(8,5))
    ax.hist(psamples[pname], bins=50, density=True)
    ax.vlines(true_values[pname], 0, 1, transform=ax.get_xaxis_transform(), linestyles='dashed', color="C2")
    plt.tight_layout()
    plt.show()

### Convergence analysis

In [ ]:
epsr = getEPSR(mcmc_ehmm_bp.samples, ehmm_burnin)

In [ ]:
from matplotlib.ticker import FormatStrFormatter
fig, ax = plt.subplots(figsize=(8,5))
xs = np.array(range(1, T+1))
for i in range(n):
    ax.plot(xs, epsr[:, i])
ax.set_xmargin(0.01)
ax.locator_params(axis='y', nbins=10)
ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
# for i, tick in enumerate(ax.yaxis.get_ticklabels()):
#     if i % 2 != 0:
#         tick.set_visible(False) 
plt.tight_layout()
# plt.savefig(OUTPUTS_PATH/"epsr_met_lp.png", dpi=300, format='png')
plt.show();

### Autocorrelation

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))
xs = np.array(range(1, T+1))
for i in range(n):
    ax.plot(xs, taus_ehmm[i, :]*overview.loc['Time per sample, ms', 'EHMM on Poisson model 2'])
ax.set_xmargin(0.01)
ax.set_xlabel("time $t$")
ax.set_ylabel("autocorrelation time, ms")
plt.tight_layout()
# plt.savefig(OUTPUTS_PATH/"taus_ehmm_lp.png", dpi=300, format='png')
plt.show();

In [ ]:
t = 0
d = 0

fig, ax = plt.subplots(figsize=(8,5))
plotACF(ac_ehmm, t, d, ax)
ax.set_ylim(-0.25,1)
plt.tight_layout()
# plt.savefig(OUTPUTS_PATH/f"acf_ehmm_{t}-{d}_lp.png", dpi=300, format='png')
plt.show();

### Trace plots

In [ ]:
plot_trace(ehmm_samples, t, d)

In [ ]:
plot_mixing(mcmc_ehmm_bp.samples, t, d)

In [ ]:
for idx, param_name in enumerate(params_names):
    fig, ax = plt.subplots(figsize=(8, 5))

    for seed in seeds:
        data = mcmc_ehmm_bp.get_paramSamples(0, params_names, forseed=seed)
        ax.plot(data.index, data[param_name], label=f"seed {seed}")

    ax.set_xmargin(0.01)
    ax.set_ylabel(param_name)
    ax.set_xlabel("sample index")
    ax.legend()
    plt.tight_layout()
    # plt.savefig(OUTPUTS_PATH / f"param_plotmix_{param_name}.png", dpi=300, format='png')
    plt.show()